In [ ]:
import cv2
import numpy as np
from skimage.morphology import skeletonize
from google.colab.patches import cv2_imshow
import os
import math
import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#get all the images from the directory
def extract_image_list(directory):
  current_directory = directory
  all_images_names = []
  for filename in os.listdir(current_directory):
    if ('.jpg' or '.png' or '.jpeg') in filename:
      all_images_names.append(filename)
  return all_images_names

In [ ]:
#for creating a output.txt which will contain coordinates of hands angles and time of certain clock
def write_to_file(file_path, content):
  try:
    with open(file_path, 'w') as file:
      file.write(content)
      print("{} Created".format(file_path))
  except Exception as e:
    print("An error occurred:", e)

def append_to_file(file_path, content):
  try:
    with open(file_path, 'a') as file:
      file.write(content)
    print("writing.........")
  except Exception as e:
    print("An error occurred while appending to the file:", e)

In [ ]:
def thresholded_image(image):

  hh, ww = image.shape[:2]

  # convert to gray
  gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

  # threshold
  thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)[1]

  # invert so shapes are white on black background
  thresh = 255 - thresh

  return thresh

In [ ]:
def detect_arms(thresh): #thresholded image
  # get contours and save area
  cntrs_info = []
  contours = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  contours = contours[0] if len(contours) == 2 else contours[1]
  index=0
  for cntr in contours:
    area = cv2.contourArea(cntr)
    cntrs_info.append((index,area))
    index = index + 1

  # sort contours by area
  def takeSecond(elem):
    return elem[1]
  cntrs_info.sort(key=takeSecond, reverse=True)

  # get third largest contour
  arms = np.zeros_like(thresh)
  index_third = cntrs_info[2][0]
  cv2.drawContours(arms,[contours[index_third]],0,(1),-1)

  #(255*arms).clip(0,255).astype(np.uint8) # better visualizations

  #arms thinning
  #arms=cv2.ximgproc.thinning(arms)
  arms_thin = skeletonize(arms)
  arms_thin = (255*arms_thin).clip(0,255).astype(np.uint8)

  return (255*arms).clip(0,255).astype(np.uint8), arms_thin

In [ ]:
def twelveOclock(result, thresh_img):
  # Find contours in the thresholded image
  contours, _ = cv2.findContours(thresh_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Assuming the largest contour is the clock face, find its center and radius
  largest_contour = max(contours, key=cv2.contourArea)
  (x, y), radius = cv2.minEnclosingCircle(largest_contour)
  radius = radius
  center = (int(x), int(y))
  radius = int(radius)

  # Calculate coordinates of 12 o'clock position
  x_12 = center[0]
  y_12 = center[1] - radius

  # Draw a circle around the clock face
  cv2.circle(result, center, radius, (0, 255, 0), 2)

  # Draw a point at the 12 o'clock position
  cv2.circle(result, (x_12, y_12), 5, (0, 0, 255), -1)

  return result, (x_12, y_12), center

In [ ]:
def angle_between_lines(line1, line2):
    # Convert lines to vectors
    vector1 = np.array([line1[2] - line1[0], line1[3] - line1[1]])
    vector2 = np.array([line2[2] - line2[0], line2[3] - line2[1]])

    # Calculate dot product and magnitudes
    dot_product = np.dot(vector1, vector2)
    magnitude1 = np.linalg.norm(vector1)
    magnitude2 = np.linalg.norm(vector2)

    # Calculate angle in radians
    theta_rad = np.arctan2(vector2[1], vector2[0]) - np.arctan2(vector1[1], vector1[0])
    #print(np.arctan2(vector2[1], vector2[0]))
    #print(np.arctan2(vector1[1], vector1[0]))
    # Convert angle to degrees
    theta_deg = theta_rad * 180 / np.pi

    # Ensure angle is between 0 and 360 degrees
    if theta_deg < 0:
        theta_deg += 360

    return theta_deg

In [ ]:
#find the length of each hand help in there positions
def length_of_hands(x):
    return math.sqrt((x[0] - x[2])**2 + (x[1] - x[3])**2)

In [ ]:
 #TODO
  #Final taks is to compute the time with respect to angle of each hand and 12 o'clock line
def calculate_time(angle1, angle2):
  #angle1 is angle formed by big hand, angle2 is angle formed by small hand
  angle_per_min = 6.0 #(360degree/60min)
  minutes = angle1/angle_per_min
  hours = angle2/30
  return hours, minutes

In [ ]:
def analog_to_digital(path, write_directory, image_name):
  # Read image
  img = cv2.imread(path + image_name) #best 26.jpg
  thresh_img = thresholded_image(img)
  cv2.imwrite(write_directory + "thresholded/" + image_name, thresh_img)
  actual, thin = detect_arms(thresh_img)
  cv2.imwrite(write_directory + "actual_arms/" + image_name, actual)
  cv2.imwrite(write_directory + "thin_arms/" + image_name, thin)
  # get hough lines and draw on copy of input
  result = img.copy()
  lineThresh = 15
  minLineLength = 20
  maxLineGap = 100
  lines = cv2.HoughLinesP(thin, 1, np.pi/180, lineThresh, None, minLineLength, maxLineGap)
  for [line] in lines:
    x1 = line[0]
    y1 = line[1]
    x2 = line[2]
    y2 = line[3]
    cv2.line(result, (x1,y1), (x2,y2), (0,0,255), 2)
  cv2.imwrite(write_directory + "colored_hands/" + image_name, result)
  circle_image, twelve_coord, cent = twelveOclock(result, thresh_img)

  # Save the image with the detected center and 12 o'clock position
  cv2.imwrite(write_directory + "circled/" + image_name, circle_image)

  # write coordinates of 12 o'clock position in output.txt
  append_to_file(write_directory + "output.txt", "Coordinates of center of clock : {} \n".format(cent))
  append_to_file(write_directory + "output.txt", "Coordinates of 12 o'clock : {} \n".format(twelve_coord))

  if(length_of_hands(lines[0][0]) > length_of_hands(lines[1][0])):
    big_hand = [cent[0], cent[1], lines[0][0][2], lines[0][0][3]]
    small_hand = [cent[0], cent[1], lines[1][0][2], lines[1][0][3]]
  else:
    big_hand = [cent[0], cent[1], lines[1][0][2], lines[1][0][3]]
    small_hand = [cent[0], cent[1], lines[0][0][2], lines[0][0][3]]
  perp = np.array([cent[0], cent[1], twelve_coord[0], twelve_coord[1]])
  angle_big = angle_between_lines(perp, big_hand)
  append_to_file(write_directory + "output.txt", "Angle between the 12 o' clock perpendicular line and Big Hand : {} \n".format(angle_big))
  angle_small = angle_between_lines(perp, small_hand)
  append_to_file(write_directory + "output.txt", "Angle between the 12 o' clock perpendicular line and Small Hand : {} \n".format(angle_small))

  h, m = calculate_time(angle_big, angle_small)
  append_to_file(write_directory + "output.txt", "Actual accurate time is {}hrs   :  {}min\n".format(round(h, 2), round(m, 2)))
  if (int(h) == 0): h = 12
  append_to_file(write_directory + "output.txt", "Time is {}:{} min\n".format(int(h), int(round(m, 2))))
  append_to_file(write_directory + "output.txt", "-"*120 + "\n")


In [ ]:
def main():
  #images read and write directories
  image_directory = "/content/drive/MyDrive/CV Project/analog_clock" + "/"
  write_directory = "/content/drive/MyDrive/CV Project" + "/"

  #get list of images from image_directory
  image_list = extract_image_list(image_directory)

  #creating result directory based on current time
  current_time = datetime.datetime.now()
  write_directory = os.path.join(write_directory, "Results_" + str(current_time) + "/")
  os.mkdir(write_directory)

  #write extracted image in output.txt
  write_to_file(write_directory + "output.txt", "Number of Images in directory = {} \n".format(len(image_list)))
  append_to_file(write_directory + "output.txt", "Extracted Images Names : {} \n".format(image_list))
  append_to_file(write_directory + "output.txt", "-"*120 + "\n")

  #create_directory to write output images
  folders = ["circled", "actual_arms", "colored_hands", "thin_arms", "thresholded"]
  for folder_name in folders:
    if(os.path.exists(write_directory + folder_name) != True):os.mkdir(write_directory + folder_name)
  #works on only these clocks among 75 clocks

  dummy_image_list = ['25.jpg', '89-2.jpg', '89.jpg', '87.jpg', '24.jpg', '84.jpg', '51.jpg', '86.jpg', '76.jpg', '92.jpg', '68.jpg', '26.jpg', '8-2.jpg', '97-2.jpg', '92-2.jpg', '76-2.jpg', '97-3.jpg', '8-3.jpg', '8-4.jpg', '21-2.jpg', '86-2.jpg', '97-4.jpg', '92-3.jpg', '92-5.jpg', '8.jpg', '10.jpg', '97.jpg']
  for image in dummy_image_list:
    append_to_file(write_directory + "output.txt", "{}\n".format(image))
    analog_to_digital(image_directory, write_directory, image)

In [ ]:
main()

/content/drive/MyDrive/CV Project/Results_2024-04-27 15:47:23.016179/output.txt Created
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing.........
writing....

In [ ]:
#Awesome Loading animation
import time
import sys
i = 0
for j in range(25):
  i = i+1
  b = ("loading" + "."*(i%10))
  sys.stdout.write('\r'+b)
  time.sleep(1)
sys.stdout.write("\r" + "Loading Complete")

Loading Complete